In [1]:
import open3d as o3d
import numpy as np 
import pandas as pd
import trimesh
import matplotlib.pyplot as plt
from  pye57 import E57
from pyntcloud import PyntCloud

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
e57 = E57('office.e57')

# get point cloud
data = e57.read_scan(0, colors=True)    

# Extract point coordinates, colors, and normals
xyz = np.column_stack((data['cartesianX'], data['cartesianY'], data['cartesianZ']))
rgb = np.column_stack((data['colorRed'], data['colorGreen'], data['colorBlue']))    

# Normalize colors to the range [0, 1]
rgb = rgb / 255.0

#creaet new point cloud in Open3D library
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(rgb)

# print
print(pcd)
print(np.asarray(pcd.points))
#o3d.visualization.draw_geometries([pcd])


PointCloud with 4562677 points.
[[ 0.55695656 -0.33749982  0.0069483 ]
 [ 0.55747682 -0.33654353  0.00691749]
 [ 0.55755344 -0.33531978  0.00774316]
 ...
 [ 0.07412872 -0.00658993  2.5882676 ]
 [ 0.07422355 -0.00646523  2.58826377]
 [ 0.07431816 -0.00634036  2.58825981]]


In [2]:
#creaet new point cloud in Open3D library
pcd = o3d.io.read_point_cloud('flattened_e57.pcd')

# print
print(pcd)
print(np.asarray(pcd.points))

PointCloud with 5511887 points.
[[-0.85553586 -4.20415354  2.05446649]
 [-0.29681262  3.10660315  0.0195704 ]
 [ 9.64883232 -2.65889239  2.25809789]
 ...
 [-0.98308057 -0.11858951  2.5919838 ]
 [-1.0954423  -0.0985795   2.59227586]
 [-1.13184023 -0.21692982  2.5907495 ]]


In [ ]:
# visualization
o3d.visualization.draw_geometries([pcd])

In [3]:
#  Downscale voxel

print("Downsampling")
downpcd = pcd.voxel_down_sample(voxel_size=.05)


Downsampling


In [ ]:
# visualization
o3d.visualization.draw_geometries([downpcd])

In [4]:
print("Best Attempt at creating normals")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))


Best Attempt at creating normals


In [5]:
# visualization
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

In [30]:
print("DBScan Clustering")

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps=0.025, min_points=10, print_progress=True))
    
max_label = labels.max()

print (f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))

colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])


DBScan Clustering
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2698
point cloud has 2698 clusters


In [29]:
o3d.visualization.draw_geometries([pcd])

In [16]:
print("attempt segmentation")
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)

[a,b,c,d] = plane_model
print(f"Plane Equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0,0,0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)

                                    


attempt segmentation
Plane Equation: 0.00x + -0.00y + 1.00z + -0.00 = 0


In [17]:
# visualization
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])